지역
- 김포시
- A군, 수요 데이터 있음

중심점
- 행복주택

고려한 요소들
- 공영주차장
- 쏘카존
- 지하철

스케일링
- 하지 않음

In [1]:
import pandas as pd
import numpy as np
import dataframe_image as dfi

In [2]:
# 데이터프레임을 그림 파일로 저장
save_option = 1 # 1 : dfi / 2 : df2img / 3 : do nothing
import df2img
def save(df, path):
    if save_option == 1:
        dfi.export(df, path)
    if save_option == 2:
        df2img.save_dataframe(fig=df2img.plot_dataframe(df), filename=path)
    if save_option == 3:
        pass

각 군집별 도시들
- cluster A : ['고양시 덕양구', '김포시', '성남시 분당구', '시흥시', '용인시 기흥구', '의정부시', '파주시']
- cluster B : ['구리시', '군포시', '성남시 수정구', '성남시 중원구', '수원시 장안구', '수원시 팔달구', '안성시', '안양시 만안구', '양주시', '오산시', '용인시 처인구', '의왕시', '이천시']
- cluster C : ['남양주시', '부천시', '화성시']
- cluster D : ['가평군', '과천시', '동두천시', '양평군', '여주시', '연천군', '포천시']
- cluster E : ['고양시 일산동구', '고양시 일산서구', '광명시', '광주시', '수원시 권선구', '수원시 영통구', '안산시 단원구', '안산시 상록구', '안양시 동안구', '용인시 수지구', '하남시']
- cluster F : ['평택시']

각 군집별 (수요 데이터가 있는) 도시들
- Cluster A : ['고양시 덕양구', '김포시', '성남시 분당구', '의정부시']
- Cluster B : ['구리시', '성남시 수정구', '성남시 중원구', '안양시 만안구']
- Cluster C : ['남양주시', '화성시']
- Cluster D : ['양평군']
- Cluster E : ['고양시 일산동구', '고양시 일산서구', '광명시', '안양시 동안구', '하남시']
- Cluster F : []

김포시는 A군에 속해있고, 수요데이터도 있는 도시이다.

# 데이터 작업

## 행복주택

In [3]:
happys = pd.read_csv("../../data/data_b_raw/2b206_0111_경기도 행복주택주거지현황.csv")
happys.rename(columns={'주소':'행복주택 주소'}, inplace=True)
happys = happys[happys['지역']=='김포시'].reset_index(drop=True)
happy_names = happys[['행복주택 주소']]
happys = happys[['경도', '위도']].rename(columns={'경도':'x', '위도':'y'})
happys = happys.values
happys.shape

(7, 2)

## 공영주차장

In [4]:
parks = pd.read_csv("../../data/data_b_raw/2b208_0202_한국교통안전공단_전국공영주차장정보_20191224.csv", encoding='CP949')
parks.rename(columns={'주차장지번주소':'주차장 주소'}, inplace=True)
parks = parks[parks['지역구분']=='경기도']
parks = parks[parks['지역구분_sub']=='김포시'].reset_index(drop=True)
park_names = parks[['주차장 주소']]
parks = parks[['경도', '위도']].rename(columns={'경도':'x','위도':'y'})
parks = parks.values
parks.shape

(38, 2)

## 쏘카존

In [5]:
# 쏘카존 갯수 데이터(김선중)
# 출처 : https://d3s0tskafalll9.cloudfront.net/media/documents/20210823_socar_zone.csv (going deeper 4-6)
zones = pd.read_csv("../../data/data_b_raw/2b201_1228_20210823_socar_zone.csv")
zones.rename(columns={'zone_name':'존 이름'}, inplace=True)
zones = zones[zones['region2']=='김포시'].reset_index(drop=True)
zone_names = zones[['존 이름']]
zones = zones[['lng','lat']]
zones.rename(columns={'lng':'x','lat':'y'}, inplace=True)
zones = zones.values
zones.shape

(55, 2)

## 지하철역

In [6]:
subways = pd.read_csv("../../data/data_b_raw/2b207_0202_서울시 역사마스터 정보.csv", encoding='CP949')
subways.rename(columns={'역사명':'역 이름'}, inplace=True)
subways = subways[subways['호선']=='김포골드라인'].reset_index(drop=True)
subways = subways[1:]
subway_names = subways[['역 이름']]
subways = subways[['위도', '경도']].rename(columns={'위도':'x','경도':'y'})
subways = subways.values

# 제안

In [7]:
n_happys = len(happys)
n_parks = len(parks)
n_zones = len(zones)
n_subways = len(subways)
print("n_happyhouses :",n_happys)
print("n_parks :",n_parks)
print("n_zones :",n_zones)
print("n_subways :",n_subways)

n_happyhouses : 7
n_parks : 38
n_zones : 55
n_subways : 9


In [8]:
# euclidean distance
def d(x:np.array, y:np.array) -> float:
    return np.linalg.norm(x-y)
x = np.array([0,3])
y = np.array([4,0])
d(x,y)

5.0

## score 설정

임의의 위치 $X$에 대하여, 다음과 같은 사실들을 가정한다.
- 가정1 : $X$에서 공영주차장이 가까이 위치할 수록 $X$의 근처에 쏘카존을 제안할 만하다. [subscript : P]
- 가정2 : $X$에서 쏘카존이 멀리 위치할 수록 $X$의 근처에 쏘카존을 제안할 만하다. [subscript : Z]
- 가정3 : $X$에서 지하철역이 멀리 위치할 수록 $X$의 근처에 쏘카존을 제안할 만하다. [subscript : S]
(지하철이 있으면, 지하철을 타지 쏘카를 타지는 않을 것이다.)
- 가정3 : $X$의 근처에 아파트가 많을수록 $X$의 근처에 쏘카존을 제안할 만하다. [subscript : A]
(상관분석 결과 반영)

위치 $X$에 대하여, 해당 위치 근처에 쏘카존을 제안할 만한지를 나타내는 score (score function)를 $f(X)$라고 하자.
<!-- (일반적인 목적함수 e.g. loss function/cost function/error function는 목적함수의 값을 최소화하는 것이 목적이지만, 이번 목적함수는 최대화하는 것이 목적이다.) -->

\begin{align*}
f(X)&=\alpha_P f_P(X) + \alpha_Z f_Z(X) + \alpha_S f_S(X) + \alpha_A f_A(X)\\
f_P(X)&:-(\text{가장 가까운 공영주차장까지의 거리})\\
f_Z(X)&:(\text{가장 가까운 쏘카존까지의 거리})\\
f_S(X)&:(\text{가장 가까운 지하철역까지의 거리})\\
f_A(X)&:(\text{반경 1km 이내의 아파트의 개수})
\end{align*}

단, $\alpha_P$, $\alpha_Z$, $\alpha_S$, $\alpha_A$는 각각의 additive subfunction에 대한 가중치로, 해당 feature들을 어느 정도의 중요도로 반영할지를 나타내는 값이다.
가중치가 모두 1이면, score는 다음과 같은 비교적 간단한 식이 된다.

\begin{align*}
f(X)
= &-(\text{가장 가까운 공영주차장까지의 거리}) + (\text{가장 가까운 쏘카존까지의 거리})\\
&+ (\text{가장 가까운 지하철역까지의 거리}) + (\text{반경 1km 이내의 아파트의 개수})
\end{align*}

## 계산

In [9]:
# distances : the distances of the happy house and every parking lot
# park_info : the list of (min,index) pair for every happy houses.
parks_info = []
for happy in happys:
    distances = [d(happy, park) for park in parks]
    parks_info.append(np.array([np.min(distances),np.argmin(distances)]))
parks_info = np.array(parks_info)
parks_info

array([[7.02302362e-03, 3.20000000e+01],
       [8.32297016e-03, 3.50000000e+01],
       [3.90377479e-02, 3.20000000e+01],
       [7.75676226e-03, 3.50000000e+01],
       [4.09918655e-02, 3.20000000e+01],
       [8.05112399e-03, 3.20000000e+01],
       [4.73909828e-02, 3.20000000e+01]])

In [10]:
parks_info_ = pd.DataFrame({'주차장 최소거리':parks_info[:,0], 'index':np.round(parks_info[:,1]).astype(int)})
parks_info_['주차장 주소'] = parks_info_['index'].replace(park_names['주차장 주소'].to_dict())
parks_info_

,주차장 최소거리,index,주차장 주소
0,0.007023,32,경기도 김포시 장기동 1796
1,0.008323,35,경기도 김포시 통진읍 서암리 756-8번지
2,0.039038,32,경기도 김포시 장기동 1796
3,0.007757,35,경기도 김포시 통진읍 서암리 756-8번지
4,0.040992,32,경기도 김포시 장기동 1796
5,0.008051,32,경기도 김포시 장기동 1796
6,0.047391,32,경기도 김포시 장기동 1796


In [11]:
# distances : the distances of the happy house and every zone
# zone_info : the list of (min,index) pair for every happy houses.
zones_info = []
for happy in happys:
    distances = [d(happy, zone) for zone in zones]
    zones_info.append(np.array([np.min(distances),np.argmin(distances)]))
zones_info = np.array(zones_info)
zones_info

array([[1.82318331e-03, 8.00000000e+00],
       [8.78407592e-03, 4.00000000e+01],
       [2.62267413e-03, 3.10000000e+01],
       [8.75833728e-03, 4.00000000e+01],
       [3.59447557e-03, 3.10000000e+01],
       [4.17918161e-03, 8.00000000e+00],
       [3.04129485e-03, 3.70000000e+01]])

In [12]:
zones_info_ = pd.DataFrame({'쏘카존 최소거리':zones_info[:,0], 'index':np.round(zones_info[:,1]).astype(int)})
zones_info_['존 이름'] = zones_info_['index'].replace(zone_names['존 이름'].to_dict())
zones_info_

,쏘카존 최소거리,index,존 이름
0,0.001823,8,중흥S클래스 리버티아파트 318동
1,0.008784,40,김포통진 성원프라자
2,0.002623,31,곡촌마을 앞
3,0.008758,40,김포통진 성원프라자
4,0.003594,31,곡촌마을 앞
5,0.004179,8,중흥S클래스 리버티아파트 318동
6,0.003041,37,메가박스 김포한강신도시점


In [13]:
# distances : the distances of the happy house and every subway
# subway_info : the list of (min,index) pair for every happy houses.
subways_info = []
for happy in happys:
    distances = [d(happy, subway) for subway in subways]
    subways_info.append(np.array([np.min(distances),np.argmin(distances)]))
subways_info = np.array(subways_info)
subways_info

array([[7.24102738e-03, 5.00000000e+00],
       [4.66810068e-02, 8.00000000e+00],
       [1.36764503e-02, 7.00000000e+00],
       [4.76031302e-02, 8.00000000e+00],
       [1.53283998e-02, 7.00000000e+00],
       [8.68483737e-03, 5.00000000e+00],
       [7.46409666e-03, 8.00000000e+00]])

In [14]:
subways_info_ = pd.DataFrame({'지하철 최소거리':subways_info[:,0], 'index':np.round(subways_info[:,1]).astype(int)})
subways_info_['역 이름'] = subways_info_['index'].replace(subway_names['역 이름'].to_dict())
subways_info_

,지하철 최소거리,index,역 이름
0,0.007241,5,운양
1,0.046681,8,구래
2,0.013676,7,마산
3,0.047603,8,구래
4,0.015328,7,마산
5,0.008685,5,운양
6,0.007464,8,구래


In [15]:
parks_info_.drop('index', axis=1), zones_info_.drop('index', axis=1), subways_info_.drop('index', axis=1)

(   주차장 최소거리                   주차장 주소
 0  0.007023         경기도 김포시 장기동 1796
 1  0.008323  경기도 김포시 통진읍 서암리 756-8번지
 2  0.039038         경기도 김포시 장기동 1796
 3  0.007757  경기도 김포시 통진읍 서암리 756-8번지
 4  0.040992         경기도 김포시 장기동 1796
 5  0.008051         경기도 김포시 장기동 1796
 6  0.047391         경기도 김포시 장기동 1796,
    쏘카존 최소거리                존 이름
 0  0.001823  중흥S클래스 리버티아파트 318동
 1  0.008784          김포통진 성원프라자
 2  0.002623              곡촌마을 앞
 3  0.008758          김포통진 성원프라자
 4  0.003594              곡촌마을 앞
 5  0.004179  중흥S클래스 리버티아파트 318동
 6  0.003041      메가박스 김포한강신도시점 ,
    지하철 최소거리 역 이름
 0  0.007241   운양
 1  0.046681   구래
 2  0.013676   마산
 3  0.047603   구래
 4  0.015328   마산
 5  0.008685   운양
 6  0.007464   구래)

In [20]:
scores = pd.concat([happy_names, parks_info_.drop('index', axis=1), zones_info_.drop('index', axis=1), subways_info_.drop('index', axis=1)], axis=1)
scores['스코어'] = -scores['주차장 최소거리'] + scores['쏘카존 최소거리'] + scores['지하철 최소거리']
scores.sort_values(by='스코어', ascending=False, inplace=True)
scores = scores.rename(columns={'주차장 주소':'가까운 주차장 주소','존 이름':'가까운 존 이름','역 이름':'가까운 역 이름'})
scores

,행복주택 주소,주차장 최소거리,가까운 주차장 주소,쏘카존 최소거리,가까운 존 이름,지하철 최소거리,가까운 역 이름,스코어
3,경기도 김포시 통진읍 율마로450번길 51,0.007757,경기도 김포시 통진읍 서암리 756-8번지,0.008758,김포통진 성원프라자,0.047603,구래,0.048605
1,경기도 김포시 통진읍 율마로438번길 51,0.008323,경기도 김포시 통진읍 서암리 756-8번지,0.008784,김포통진 성원프라자,0.046681,구래,0.047142
5,경기도 김포시 김포한강3로237번길 33,0.008051,경기도 김포시 장기동 1796,0.004179,중흥S클래스 리버티아파트 318동,0.008685,운양,0.004813
0,경기도 김포시 허산길 50,0.007023,경기도 김포시 장기동 1796,0.001823,중흥S클래스 리버티아파트 318동,0.007241,운양,0.002041
4,경기도 김포시 양촌읍 양곡4로197번길 11,0.040992,경기도 김포시 장기동 1796,0.003594,곡촌마을 앞,0.015328,마산,-0.022069
2,경기도 김포시 양촌읍 양곡2로80번길 61,0.039038,경기도 김포시 장기동 1796,0.002623,곡촌마을 앞,0.013676,마산,-0.022739
6,경기도 김포시 김포한강4로 564,0.047391,경기도 김포시 장기동 1796,0.003041,메가박스 김포한강신도시점,0.007464,구래,-0.036886


In [21]:
save(scores, "results/1d201_0202_scores_김포.png")